In [1]:
import os

inputs = []
label = []

dirts = 'ws_data'

for dirt in os.listdir(dirts):
    files_path = os.path.join(dirts, dirt)
    for file in os.listdir(files_path):
        text = ''
        file_path = os.path.join(files_path, file)
        with open(file_path, 'r', errors='ignore') as f:
            for l in f:
                text = l + ' '

        if text.strip() != '':
            inputs.append(text)
            label.append(dirt)

### Preprocessing

In [2]:
import pandas as pd

docs = pd.DataFrame()
docs['text'] = inputs
docs['class'] = label
docs

,text,class
0,As is clear to anyone who pays medical insuran...,Health
1,The health insurance system in the United Stat...,Health
2,COVID-19’s impacts on workers and workplaces a...,Health
3,"Months of binge watching, doom scrolling, home...",Health
4,All organizations now practice some form of ri...,Health
...,...,...
1131,When we wonder whether money can buy happiness...,Finance
1132,With executive compensation soaring to unprece...,Finance
1133,Credential of Readiness (CORe) program launche...,Finance
1134,The duration of a vertical relationship depend...,Finance


### Extracting Features

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

stop_words = text.ENGLISH_STOP_WORDS.union()
count_vect = CountVectorizer(token_pattern="[^\W\d_]+")
train_data_features = count_vect.fit_transform(docs['text'])

def preprocess_file(file_path):
    text = ''
    with open(file_path, 'r', errors='ignore') as f:
        for l in f:
            text = l + ' '
    return count_vect.transform([text])

def preprocess(input):
    return count_vect.transform(input)

### Using Multinomial Naive Bayes Model from sklearn

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

docs['class'] = docs['class'].replace(docs['class'].unique().tolist(), range(0, len(docs['class'].unique())))
X = train_data_features.toarray()
Y = docs['class'].tolist()

x, _x, y, _y = train_test_split(X, Y, test_size=0.1, random_state=42)

clf = MultinomialNB()
_ = clf.fit(x, y)

### Testing the Model

In [5]:
from sklearn.metrics import confusion_matrix

yhat = clf.predict(_x)
matrix = confusion_matrix(_y, yhat)
accuracy = (matrix.diagonal() / matrix.sum(axis=1)) * 100

acc_table = pd.DataFrame()
acc_table['category'] = docs['class'].unique()
acc_table['accuracy (%)'] = accuracy.round(1)

acc_table

,category,accuracy (%)
0,0,73.7
1,1,56.9
2,2,86.4
